# 10 minutes to cuxfilter

This is a short introduction to the cuxfilter.py library, mostly going over the basic usage and features provided as a quick summary.

### What is cuxfilter?

cuxfilter is inspired from the Crossfilter library, which is a fast, browser-based filtering mechanism across multiple dimensions and offers features do groupby operations on top of the dimensions. One of the major limitations of using Crossfilter is that it keeps data in-memory on a client-side browser, making it inefficient for processing large datasets.

cuxfilter solves the issues by leveraging the power of the rapids.ai stack, mainly cudf. The data is maintained in a gpu as a GPU DataFrame and operations like groupby aggregations, sorting and querying are done on the gpu itself, only returning the result as the output to the charts.

cuxfilter acts as a `connector` library, which provides the connections between different visualization libraries and a GPU dataframe without much hassle. This also allows the user to use charts from different libraries in a single dashboard, while also providing the interaction.

### The modules

> cuxfilter has following usable modules

1. cuxfilter.DataFrame
2. cuxfilter.DashBoard
3. cuxfilter.charts
4. cuxfilter.layouts
5. cuxfilter.themes

### Usage

#### 1. Import the required modules

In [1]:
import cuxfilter
from cuxfilter import DataFrame, themes, layouts

#### Download required datasets

In [3]:
#update data_dir if you have downloaded datasets elsewhere
DATA_DIR = './data/'

! curl https://data.rapids.ai/viz-data/auto_accidents.arrow.gz --create-dirs -o $DATA_DIR/auto_accidents.arrow.gz

In [4]:
from cuxfilter.sampledata import datasets_check
datasets_check('auto_accidents', base_dir=DATA_DIR)

Dataset - ./data//auto_accidents.arrow

dataset already downloaded


#### 2. Read some data

> cuxfilter can read arrow files off disk, or an inmemory cudf dataframe

In [5]:
#create cuxfilter DataFrame
cux_df = DataFrame.from_arrow(DATA_DIR + './auto_accidents.arrow')
cux_df.data['ST_CASE'] = cux_df.data['ST_CASE'].astype('float64')

# add a day_week_str column, using cudf.Series.map()
cux_df.data['DAY_WEEK_STR'] = cux_df.data.DAY_WEEK.map({1: 'Sunday',    2: 'Monday',    3: 'Tuesday',    4: 'Wednesday',   5: 'Thursday',    6: 'Friday',    7: 'Saturday',    9: 'Unknown'})

cux_df.data[['dropoff_x', 'dropoff_y', 'DAY_WEEK', 'DAY_WEEK_STR', 'YEAR', 'MONTH']].head()

,dropoff_x,dropoff_y,DAY_WEEK,DAY_WEEK_STR,YEAR,MONTH
__index_level_0__,,,,,,
0,-9.685585e+06,3.939943e+06,1,Sunday,2017,2
1,-9.661068e+06,4.117979e+06,3,Tuesday,2017,2
2,-9.589649e+06,3.811519e+06,3,Tuesday,2017,1
3,-9.589649e+06,3.811519e+06,3,Tuesday,2017,1
4,-9.589649e+06,3.811519e+06,3,Tuesday,2017,1


#### 3. Create some charts

> see charts section to see available chart options

In [ ]:
demo_red_blue_palette = [ "#3182bd", "#6baed6", "#7b8ed8", "#e26798", "#ff0068" , "#323232" ]

chart1 = cuxfilter.charts.scatter(x='dropoff_x', y='dropoff_y', aggregate_col='DAY_WEEK', aggregate_fn='mean',
                                color_palette=demo_red_blue_palette, tile_provider='CartoLight',
                                pixel_shade_type='linear')
chart2 = cuxfilter.charts.bar('YEAR')


chart3 = cuxfilter.charts.multi_select('DAY_WEEK_STR')

chart4 = cuxfilter.charts.number(expression="AGE", aggregate_fn="mean", title="Mean age")
chart5 = cuxfilter.charts.number(expression="SIDE_DRIV_STARS + FRNT_DRIV_STARS", aggregate_fn="mean", title="Vehicle(Mean front+side safety rating)")
charts_list = [chart1, chart2 ]
sidebar = [chart3, chart4, chart5]

#### 4. Create a dashboard object 

In [ ]:
d = cux_df.dashboard(charts_list, sidebar=sidebar, title='Auto Accidents Dashboard', layout=layouts.feature_and_base, theme=themes.default)

#### 5. Run the dashboard

In [ ]:
"""
1. d.show('current_notebook_url:current_notebook_port', sidebar_width=300, height=1000) remote dashboard

2. d.app() inline within the notebook cell

Incase you need to stop the server:

- d.stop()
"""

d.app(width=1000, height=800)

![auto-accidents-notebook](../../_images/auto-accidents-notebook.png)

In [ ]:
d.show()

![dashboard-show](../../_images/dashboard-show.png)

In [ ]:
# Will display a dashboard like the following, in a new browser tab

![auto-accidents-2](../../_images/auto-accidents-2.png)